# Part 2 - Data Visualization

## Imports

In [2]:
import pandas as pd
import numpy as np
import plotly

In [5]:
songs = pd.read_csv('songs.csv')

In [6]:
songs

,track_id,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,type_x,valence,...,album_uri,artist_id,artist_name,external_url,genre,image_url_y,followers,popularity,type_y.1,artist_uri
0,5k8kaD41vSP6l0Jhe9HzmY,0.277,0.992,0.836000,0.2720,-6.237,0.0856,103.494,audio_features,0.148,...,spotify:album:2FfewmvnA0wctMD64KjOxP,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters,https://open.spotify.com/artist/7jy3rLJdDQY21O...,alternative metal,https://i.scdn.co/image/ab6761610000e5eb9a43b8...,10203747,78,artist,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD
1,1QivlVy4J2yWaCZ6RnWl70,0.380,0.969,0.024100,0.3040,-6.147,0.0649,132.869,audio_features,0.352,...,spotify:album:2FfewmvnA0wctMD64KjOxP,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters,https://open.spotify.com/artist/7jy3rLJdDQY21O...,alternative metal,https://i.scdn.co/image/ab6761610000e5eb9a43b8...,10203747,78,artist,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD
2,58bqM9Eees0hIoem8ed3GF,0.170,0.998,0.352000,0.7570,-4.585,0.2320,162.402,audio_features,0.060,...,spotify:album:2FfewmvnA0wctMD64KjOxP,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters,https://open.spotify.com/artist/7jy3rLJdDQY21O...,alternative metal,https://i.scdn.co/image/ab6761610000e5eb9a43b8...,10203747,78,artist,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD
3,58bkJBSouJt9mMdZkYj4ND,0.111,0.978,0.760000,0.2570,-5.330,0.0737,165.373,audio_features,0.303,...,spotify:album:2FfewmvnA0wctMD64KjOxP,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters,https://open.spotify.com/artist/7jy3rLJdDQY21O...,alternative metal,https://i.scdn.co/image/ab6761610000e5eb9a43b8...,10203747,78,artist,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD
4,7ezrc9VG9RlDvRaEOt0K6D,0.253,0.949,0.008090,0.1350,-6.126,0.1110,161.611,audio_features,0.160,...,spotify:album:2FfewmvnA0wctMD64KjOxP,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters,https://open.spotify.com/artist/7jy3rLJdDQY21O...,alternative metal,https://i.scdn.co/image/ab6761610000e5eb9a43b8...,10203747,78,artist,spotify:artist:7jy3rLJdDQY21OgRLCZ9sD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4442,5ro7xAxDVbtabTl8t3MzHz,0.628,0.320,0.000351,0.2490,-16.010,0.0301,114.526,audio_features,0.545,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8604092,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
4443,0J8Q2BOEzphO2tTUlfCUln,0.600,0.449,0.027400,0.0775,-13.313,0.0374,75.103,audio_features,0.470,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8604092,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
4444,0cuiu7deGyY5kSKZgMEyaJ,0.595,0.784,0.001500,0.3500,-11.435,0.0847,115.273,audio_features,0.460,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8604092,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL
4445,40h65HAR8COEoqkMwUUQHu,0.568,0.634,0.002810,0.2520,-12.336,0.0282,142.686,audio_features,0.866,...,spotify:album:51B7LbLWgYLKBVSpkan8Z7,0ECwFtbIWEVNwjlrfc6xoL,Eagles,https://open.spotify.com/artist/0ECwFtbIWEVNwj...,album rock,https://i.scdn.co/image/ab6761610000e5eb0767e1...,8604092,75,artist,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL


In [ ]:
songs.groupby('artist_name').nunique().iloc[:,0:10]

,track_id,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,type_x,valence
artist_name,,,,,,,,,,
Adele,46,45,46,20,40,46,45,46,1,46
Bad Bunny,92,84,80,60,78,92,87,92,1,83
Beyoncé,375,257,258,218,262,325,278,332,1,275
Billie Eilish,46,45,41,43,39,44,44,46,1,46
Britney Spears,196,125,136,90,139,158,147,158,1,145
Camilo,32,32,32,5,32,32,31,32,1,32
Coldplay,209,154,169,178,155,184,139,185,1,166
Eagles,161,132,139,140,132,151,123,152,1,140
Ed Sheeran,232,117,131,45,119,145,132,150,1,135
